# Chuẩn bị dữ liệu & xử lý đặc trưng cho Titanic


## Bước 1: Load dữ liệu

In [1]:
import pandas as pd

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

print(train.shape)
train.head()


(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Bước 2: Tạo đặc trưng mới


In [2]:
train['FamilySize'] = train['SibSp'] + train['Parch'] + 1
train['IsAlone'] = (train['FamilySize'] == 1).astype(int)

# Trích xuất danh xưng (Title) từ tên
train['Title'] = train['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
train['Title'] = train['Title'].replace(['Mlle','Ms'],'Miss').replace('Mme','Mrs')


<>:5: SyntaxWarning: invalid escape sequence '\.'
<>:5: SyntaxWarning: invalid escape sequence '\.'
C:\Temp\ipykernel_16548\2082913727.py:5: SyntaxWarning: invalid escape sequence '\.'
  train['Title'] = train['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)


## Bước 3: Chọn các cột cần thiết

In [3]:
features = ['Pclass','Sex','Age','Fare','Embarked','FamilySize','IsAlone','Title']
X = train[features]
y = train['Survived']


## Bước 4: Tiền xử lý dữ liệu

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

num_cols = ['Age', 'Fare', 'FamilySize']
cat_cols = ['Sex', 'Embarked', 'Pclass', 'Title']

num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', num_pipe, num_cols),
    ('cat', cat_pipe, cat_cols)
])


## Bước 5: Fit và lưu pipeline

In [6]:
import joblib

preprocessor.fit(X)
joblib.dump(preprocessor, 'preprocessor.pkl')
print("Saved preprocessor.pkl")


Saved preprocessor.pkl


## Bước 6: Áp dụng xử lý


In [7]:
X_ready = preprocessor.transform(X)
